# Clinical Evidence Researcher Agent
In this notebook we create the clinical evidence researcher sub-agent

#### Ensure the latest version of boto3 is shown below

In [ ]:
!pip freeze | grep boto3

#### Load in environment variables to notebook

In [ ]:
# Retrieve import path
%store -r IMPORTS_PATH

# Retrieve account info
%store -r account_id
%store -r region

# Retrieve model lists
%store -r agent_foundation_model

#### Retrieve imports environment variable and bring libraries into notebook

In [ ]:
%run $IMPORTS_PATH

# Prerequisites

This notebook assumes that you have deployed the CloudFormation stack located at https://github.com/aws-samples/amazon-bedrock-agents-cancer-biomarker-discovery to your AWS account in workshop mode.

# Agent Creation
In this section we create the sub-agent

#### Define agent configuration below 

In [ ]:
agent_name = "Clinical-evidence-researcher"
agent_description = "Research internal and external evidence"
agent_instruction = """You are a medical research assistant AI specialized in summarizing internal and external 
evidence related to cancer biomarkers. Your primary task is to interpret user queries, gather internal and external 
evidence, and provide relevant medical insights based on the results. Use only the appropriate tools as required by 
the specific question. Follow these instructions carefully: 1. When querying PubMed: a. Summarize the findings of 
each relevant study with citations to the specific pubmed web link of the study b. The json output will include 
'Link', 'Title', 'Summary'. c. Always return the Title and Link (for example, 'https://pubmed.ncbi.nlm.nih.gov/') 
of each study in your response.  2. For internal evidence, make use of the knowledge base to retrieve relevant 
information. Always provide citations to specific content chunks. 3. When providing your response: 
a. Start with a brief summary of your understanding of the user's query. b. Explain the steps you're taking to 
address the query. Ask for clarifications from the user if required. c. Separate the responses generated from 
internal evidence (knowledge base) and external evidence (PubMed api).  d. Conclude with a concise summary of the 
findings and their potential implications for medical research."""

#### Instantiate agent with the desired configuration

In [ ]:
agents = AgentsForAmazonBedrock()

research_evidence_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

research_evidence_agent

#### Extract useful agent information

In [ ]:
research_evidence_agent_id = research_evidence_agent[0]
research_evidence_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{research_evidence_agent_id}"

research_evidence_agent_id, research_evidence_agent_arn

#### Associate Knowledge Base with agent and ensure that the Knowledge Base has been manually synced!

In [ ]:
bedrock_agent_client = boto3.client("bedrock-agent", region)

# Call the list_knowledge_bases method
response = bedrock_agent_client.list_knowledge_bases()

# Iterate through knowledge bases and find needed one
for kb in response['knowledgeBaseSummaries']:
    kb_name = kb['name']
    ncbi_kb_id = None

    if 'ncbiKnowledgebase' in kb_name:
        ncbi_kb_id = response['knowledgeBaseSummaries'][0]['knowledgeBaseId']

ncbi_kb_id

In [ ]:
agents.associate_kb_with_agent(
    research_evidence_agent_id,
    "Literature evidence on Relationships between Molecular and Imaging Phenotypes with Prognostic Implications", 
    ncbi_kb_id
)

#### Define the API Schema needed for an ActionGroup

In [ ]:
api_schema_string = '''{
  "openapi": "3.0.0",
  "info": {
    "title": "fetch biomedical literature",
    "version": "1.0.0", 
    "description": "PubMed API to help answer users question using abstracts from biomedical literature."
  },
  "paths": {
    "/query-pubmed": {
      "post": {
        "summary": "Query pubmed to relevant information from abstracts of biomedical articles.",
        "description": "Query pubmed to relevant information from abstracts of biomedical articles. The PubMed API takes in the user query then returns the abstracts of top 5 relevant articles.",
        "operationId": "query-pubmed",
        "parameters": [
          {
            "name": "query",
            "in": "query",
            "description": "user query",
            "required": true,
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Query pubmed to relevant information from abstracts of biomedical articles.",
            "content": {
              "application/json": {
                "schema": {
                  "type": "object",
                  "properties": {
                    "answer": {
                      "type": "string",
                      "description": "The response to user query with list of pubmed article abstracts."
                    }
                  }
                }
              }
            }
          }
        }
      }
    }
  }
}'''

In [ ]:
api_schema = {"payload": api_schema_string}

#### Attach Lambda function and create ActionGroup

In [ ]:
# Define Lambda func. details
research_evidence_lambda_function_name = "PubMedQueryFunction"
research_evidence_lambda_function_arn = f"arn:aws:lambda:{region}:{account_id}:function:{research_evidence_lambda_function_name}"
%store research_evidence_lambda_function_arn

In [ ]:
agents.add_action_group_with_lambda(
    agent_name=agent_name,
    lambda_function_name=research_evidence_lambda_function_name,
    source_code_file=research_evidence_lambda_function_arn,
    agent_action_group_name="queryPubMed",
    agent_action_group_description="Actions for fetching biomedical literature from PubMed",
    api_schema=api_schema,
    verbose=True
)

#### Add resource based policy to Lambda function to allow agent to invoke

In [ ]:
lambda_client = boto3.client('lambda', region)

# Define the resource policy statement
policy_statement = {
    "Sid": "AllowBedrockAgentAccess",
    "Effect": "Allow",
    "Principal": {
        "Service": "bedrock.amazonaws.com"
    },
    "Action": "lambda:InvokeFunction",
    "Resource": research_evidence_lambda_function_arn,
    "Condition": {
        "ArnEquals": {
            "aws:SourceArn": research_evidence_agent_arn
        }
    }
}

try:
    # Get the current policy
    response = lambda_client.get_policy(FunctionName=research_evidence_lambda_function_arn)
    current_policy = json.loads(response['Policy'])
    
    # Add the new statement to the existing policy
    current_policy['Statement'].append(policy_statement)
    
except lambda_client.exceptions.ResourceNotFoundException:
    # If there's no existing policy, create a new one
    current_policy = {
        "Version": "2012-10-17",
        "Statement": [policy_statement]
    }

# Convert the policy to JSON string
updated_policy = json.dumps(current_policy)

# Add or update the resource policy
response = lambda_client.add_permission(
    FunctionName=research_evidence_lambda_function_arn,
    StatementId="AllowResearchEvidenceAgentAccess",
    Action="lambda:InvokeFunction",
    Principal="bedrock.amazonaws.com",
    SourceArn=research_evidence_agent_arn
)

print("Resource policy added successfully.")
print("Response:", response)

#### Invoke Research Evidence Agent Test Alias to see that it answers question properly

In [ ]:
bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region)

session_id:str = str(uuid.uuid1())

test_query = "Can you search PubMed for evidence around the effects of biomarker use in oncology on clinical trial failure risk"
response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=research_evidence_agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Agent:\n{}".format(response))
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

#### Now that agent has been tested via direct invoke, prepare it by creating an alias

In [ ]:
research_evidence_agent_alias_id, research_evidence_agent_alias_arn = agents.create_agent_alias(
    research_evidence_agent[0], 'v1'
)

%store research_evidence_agent_alias_arn
research_evidence_agent_alias_id, research_evidence_agent_alias_arn